In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#import libraries

#general libraries 
import numpy as np 
import pandas as pd 
import os
import random
from sklearn.utils import shuffle
import shutil

#visualizations
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import matplotlib.patches as patches

# work with images
from skimage.transform import rotate
from skimage import io
import cv2 as cv

# model development
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import RandomFlip, RandomZoom, RandomRotation
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.optimizers import Adam

import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)

In [ ]:
tf.__version__

In [ ]:
#get files
test_path = '../input/histopathologic-cancer-detection/test/'
train_path = '../input/histopathologic-cancer-detection/train/'
sample_submission = pd.read_csv('../input/histopathologic-cancer-detection/sample_submission.csv')
train_data = pd.read_csv('../input/histopathologic-cancer-detection/train_labels.csv')

In [ ]:
# declare constants for reproduciblity
RANDOM_STATE = 370

In [ ]:
train_data.describe()

In [ ]:
train_data.info()

**EDA**

In [ ]:
label_counts = train_data['label'].value_counts()
plt.figure(figsize=(8, 6))
sns.countplot(x=train_data['label'], palette='colorblind')
plt.title('Label Counts Histogram')
plt.xlabel('Label')
plt.ylabel('Count')
plt.show()

In [ ]:
import plotly.express as px

label_counts = train_data['label'].value_counts()
fig = px.pie(train_data, 
             values=label_counts.values, 
             names=label_counts.index,
             title="Label Percentage Pie Chart")
fig.update_layout(title_x=0.5, title_y=0.95)
fig.show()

In [ ]:
fig, ax = plt.subplots(5, 5, figsize=(15, 15))

for i, axis in enumerate(ax.flat):
    if i < len(train_data):  # Ensure we don't exceed the number of images
        file = str(train_path + train_data.id[i] + '.tif')
        image = io.imread(file)
        axis.imshow(image)
        box = patches.Rectangle((16, 16), 64, 64, linewidth=2, edgecolor='r', facecolor='none')
        axis.add_patch(box)
        axis.set(xticks=[], yticks=[], xlabel=train_data.label[i], title=f'Image {i + 1}')

plt.tight_layout()
plt.show()

In [ ]:
BATCH_SIZE = 32

In [ ]:
def append_tif(string):
    return string+".tif"

train_data["id"] = train_data["id"].apply(append_tif)
train_data['label'] = train_data['label'].astype(str)


train_data = shuffle(train_data, random_state=RANDOM_STATE)

In [ ]:
datagen = ImageDataGenerator(rescale=1./255.,
                            validation_split=0.15)

In [ ]:
train_data_subset = train_data.head(8000)
train_generator = datagen.flow_from_dataframe(
    dataframe=train_data_subset,
    directory=train_path,
    x_col="id",
    y_col="label",
    subset="training",
    batch_size=BATCH_SIZE,
    seed=RANDOM_STATE,
    class_mode="binary",
    target_size=(64,64))

In [ ]:
valid_generator = datagen.flow_from_dataframe(
    dataframe=train_data_subset,
    directory=train_path,
    x_col="id",
    y_col="label",
    subset="validation",
    batch_size=BATCH_SIZE,
    seed=RANDOM_STATE,
    class_mode="binary",
    target_size=(64,64))

In [ ]:
# Setup GPU accelerator - configure Strategy. Assume TPU...if not set default for GPU/CPU
tpu = None
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
except ValueError:
    strategy = tf.distribute.get_strategy()

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, Flatten, Dense

# Set ROC AUC as metric
ROC_1 = tf.keras.metrics.AUC()

# Assuming you have a strategy defined for GPU usage
strategy = tf.distribute.MirroredStrategy()  # Define your strategy here

with strategy.scope():
    # Create model
    model_one = Sequential()
    
    model_one.add(Conv2D(filters=16, kernel_size=(3,3), input_shape=(64, 64, 3)))  # Assuming input size is (64, 64, 3)
    model_one.add(Conv2D(filters=16, kernel_size=(3,3)))
    model_one.add(MaxPooling2D(pool_size=(2,2)))

    model_one.add(Conv2D(filters=32, kernel_size=(3,3)))
    model_one.add(Conv2D(filters=32, kernel_size=(3,3)))
    model_one.add(AveragePooling2D(pool_size=(2,2)))

    model_one.add(Flatten())
    model_one.add(Dense(1, activation='sigmoid'))
    
    # Compile model
    model_one.compile(loss='binary_crossentropy', metrics=['accuracy', ROC_1])

# Quick look at model
model_one.summary()

In [ ]:
# Assuming you have defined train_generator and valid_generator for training data
EPOCHS = 5  # Reduced number of epochs

# Adjusted batch size
BATCH_SIZE = 32  # You can experiment with different batch sizes

# Train the model using `fit` with adjusted parameters
history_model_one = model_one.fit(
    train_generator,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=valid_generator
)

In [ ]:
# Plot model accuracy per epoch
plt.plot(history_model_one.history['accuracy'])
plt.plot(history_model_one.history['val_accuracy'])
plt.title('Model One Accuracy per Epoch')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot model loss per epoch
plt.plot(history_model_one.history['loss'])
plt.plot(history_model_one.history['val_loss'])
plt.title('Model One Loss per Epoch')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
#double check what you're aiming the submission data set to look like
sample_submission.head()

In [ ]:
#create a dataframe to run the predictions
test_df = pd.DataFrame({'id':os.listdir(test_path)})
test_df.head()

In [ ]:
# prepare test data (in same way as train data)
datagen_test = ImageDataGenerator(rescale=1./255.)

test_generator = datagen_test.flow_from_dataframe(
    dataframe=test_df,
    directory=test_path,
    x_col='id', 
    y_col=None,
    target_size=(64,64),         # original image = (96, 96) 
    batch_size=1,
    shuffle=False,
    class_mode=None)

In [ ]:
#run model to find predictions

# predictions = model_one.predict(test_generator, verbose=1)
predictions = model_one.predict(test_generator, verbose=1)

In [ ]:
#create submission dataframe
predictions = np.transpose(predictions)[0]
submission_df = pd.DataFrame()
submission_df['id'] = test_df['id'].apply(lambda x: x.split('.')[0])
submission_df['label'] = list(map(lambda x: 0 if x < 0.5 else 1, predictions))
submission_df.head()

In [ ]:
#convert to csv to submit to competition
submission_df.to_csv('submission.csv', index=False)